In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# Atoms
from ase.io import read
from ase.geometry import distance

# ML
from regression import SparseKRR
from kernels import build_kernel, linear_kernel, gaussian_kernel
from split import simple_split

# Utilities
from tools import load_json
from selection import FPS, random_selection
from kernels import sqeuclidean_distances
import multiprocessing
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist
from sklearn.svm import LinearSVC
import itertools
import h5py
import glob
import project_utils as utils
from tqdm.notebook import tqdm

# Plotting
import matplotlib.pyplot as plt

# SOAP
from soap import quippy_soap, librascal_soap

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
color_list = cosmostyle.color_cycle
line_list = cosmostyle.linestyle_cycle

# Initial setup

In [3]:
# Setup SOAP parameters
soap_hyperparameters = dict(max_radial=12,
                            max_angular=9,
                            interaction_cutoff=6.0,
                            cutoff_smooth_width=0.3,
                            gaussian_sigma_constant=0.3)

# Checks on geometry optimization

In [4]:
# TODO: eliminate RWY and the 5 deem structures

In [5]:
# Load indices of optimized structures
idxs_opt = np.loadtxt('../Processed_Data/Models/6.0/Linear_Models/LSVC-LPCovR/4-Class/Power/OO+OSi+SiSi/GCH/rattled/vlist.idx', dtype=int)
n_iza = 225
idxs_opt = idxs_opt[idxs_opt >= n_iza] - n_iza

In [6]:
idxs_deem_train = np.sort(np.loadtxt('../Processed_Data/DEEM_10k/idxs_bk/train.idxs', dtype=int)) ##
idxs_iza_train = np.sort(np.loadtxt('../Processed_Data/IZA_226/idxs_bk/train.idxs', dtype=int)) ##

In [7]:
cantons_iza = np.loadtxt('../Raw_Data/GULP/IZA_226/cantons.txt', usecols=1, dtype=int)
RWY = np.nonzero(cantons_iza == 4)[0][0]

In [8]:
deem_delete = np.loadtxt('../Processed_Data/DEEM_330k/10kJmol_error.idxs', dtype=int)
deem_ids = np.arange(0, 331172)
deem_ids = np.delete(deem_ids, deem_delete) + 8000001

In [26]:
# Load old SOAPs
old_soaps_deem_file = h5py.File('../Processed_Data/DEEM_330k/Data/6.0/soaps_power_full_avg_nonorm.hdf5', 'r')
old_soaps_deem = old_soaps_deem_file['0'][idxs_deem_train, :]
old_soaps_deem_opt = old_soaps_deem_file['0'][idxs_opt, :]
old_soaps_deem_file.close()

old_soaps_iza = utils.load_hdf5('../Processed_Data/IZA_226/Data/6.0/soaps_power_full_avg_nonorm.hdf5',
                                datasets=None, concatenate=True)
old_soaps_iza = np.delete(old_soaps_iza, RWY, axis=0)
old_soaps_iza = old_soaps_iza[idxs_iza_train]

old_soaps = np.vstack((old_soaps_iza, old_soaps_deem))

In [27]:
# Load old decision functions
old_df = np.loadtxt('../Processed_Data/DEEM_330k/Data/6.0/Linear_Models/LSVC-LPCovR/4-Class/Power/OO+OSi+SiSi/svc_structure_dfs.dat')[idxs_opt, :]
old_cantons = np.loadtxt('../Processed_Data/DEEM_330k/Data/6.0/Linear_Models/LSVC-LPCovR/4-Class/Power/OO+OSi+SiSi/svc_structure_cantons.dat', dtype=int)[idxs_opt]

In [28]:
# Load corresponding xyz
cif_glob = sorted(glob.glob('../Raw_Data/GULP/DEEM_330k/Geometry/8*/*_opt.cif'))
orig_cif_glob = [s.replace('Geometry/', '') for s in cif_glob]

deem_opt = [read(cif_file, format='cif') for cif_file in cif_glob]
deem_orig = [read(cif_file, format='cif') for cif_file in orig_cif_glob]

/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/ase/io/cif.py:375: UserWarning: crystal system 'triclinic' is not interpreted for space group 1. This may result in wrong setting!
  setting_name, spacegroup))


In [29]:
# Large numbers likely due to the 0/1 cell boundary
for opt, orig, glob in zip(deem_opt, deem_orig, orig_cif_glob):
    print(os.path.basename(glob), distance(opt, orig))

8129307_opt.cif 0.005578273140481637
8129502_opt.cif 23.256612886185803
8154451_opt.cif 0.016386181675007588
8162067_opt.cif 0.0010614732815458336
8171809_opt.cif 0.005712884170824324
8286860_opt.cif 0.003068503597901241
8301971_opt.cif 0.0030227870219129453


In [30]:
opt_soaps = librascal_soap(deem_opt, [14],
                           **soap_hyperparameters,
                           normalize=False, 
                           average=True, 
                           concatenate=True)

100%|██████████| 7/7 [00:00<00:00, 84.46it/s]


In [31]:
print(np.linalg.norm(opt_soaps - old_soaps_deem_opt, axis=1))

[1.63085554e-06 7.37870407e-07 7.52730881e-07 2.45267341e-07
 1.07735862e-06 1.34693209e-07 1.77601459e-07]


In [32]:
soap_scale = np.std(old_soaps)

lsvc = LinearSVC()
lsvc_dict = load_json('../Processed_Data/Models/6.0/Linear_Models/LSVC-LPCovR/4-Class/Power/OO+OSi+SiSi/svc.json', array_convert=True)
lsvc.__dict__ = lsvc_dict

opt_df = lsvc.decision_function(opt_soaps / soap_scale)
opt_cantons = lsvc.predict(opt_soaps / soap_scale)

In [33]:
opt_df

array([[ -3.43832451, -10.28983252,  -8.02390094,   4.11378737],
       [ -6.2380323 ,  -5.30778473,  -3.14313988,   2.93446246],
       [ -4.64386883, -12.86502215,  -9.81393548,   6.89819009],
       [ -6.52820209,  -5.710208  ,  -2.00586572,   0.19344744],
       [ -5.89627736, -17.98955797,  -7.07448019,   6.19365991],
       [ -3.2686013 ,  -3.32880773,  -8.45304135,   3.30751157],
       [ -4.19626375,  -2.36694088,  -5.17583915,   0.76675821]])

In [34]:
old_df

array([[ -3.60330169, -10.78943244,  -8.40059917,   4.32851384],
       [ -6.51514771,  -5.53944611,  -3.27410005,   3.06077119],
       [ -4.84927237, -13.43976356, -10.25483399,   7.21259196],
       [ -6.80901789,  -5.95515394,  -2.09624478,   0.20685544],
       [ -6.14058224, -18.79316499,  -7.40798185,   6.48128434],
       [ -3.40779344,  -3.46982879,  -8.81643724,   3.45102412],
       [ -4.37463058,  -2.47206854,  -5.40254089,   0.80355065]])

In [35]:
print((opt_df - old_df) / old_df * 100)

[[-4.57850025 -4.63045596 -4.48418286 -4.96074351]
 [-4.25340178 -4.18203136 -3.99988301 -4.12669628]
 [-4.23576    -4.276425   -4.29942124 -4.35906925]
 [-4.12417484 -4.11317559 -4.31147445 -6.48182047]
 [-3.97852958 -4.27606003 -4.50192321 -4.43776904]
 [-4.08452404 -4.0642082  -4.12179975 -4.15854971]
 [-4.07730021 -4.25261917 -4.19620591 -4.57873278]]


In [36]:
print(np.linalg.norm(opt_df - old_df, axis=1))
for idx, i in enumerate(idxs_opt):
    print(deem_ids[i], opt_cantons[idx], old_cantons[idx])

[0.6817819  0.40443096 0.81594016 0.38367196 0.94837771 0.43807969
 0.30921499]
8129309 4 4
8129504 4 4
8154453 4 4
8162069 4 4
8171811 4 4
8286863 4 4
8301974 4 4


# Check Predictions for IZA in Deem

In [21]:
#pcantons_iza = np.loadtxt('../Processed_Data/IZA_226/Data/6.0/Linear_Models/LSVC-LPCovR/2-Class/Power/OO+OSi+SiSi/svc_structure_cantons.dat', dtype=int)
pcantons_iza = np.loadtxt('../Processed_Data/IZA_226/Data/6.0/Linear_Models/LSVC-LPCovR/4-Class/Power/OO+OSi+SiSi/svc_structure_cantons.dat', dtype=int)
# Don't delete predicted! RWY has already been removed!

In [22]:
# Predicted Deem cantons
#cantons_deem = np.loadtxt('../Processed_Data/DEEM_330k/Data/6.0/Linear_Models/LSVC-LPCovR/2-Class/Power/OO+OSi+SiSi/svc_structure_cantons.dat', dtype=int)
cantons_deem = np.loadtxt('../Processed_Data/DEEM_330k/Data/6.0/Linear_Models/LSVC-LPCovR/4-Class/Power/OO+OSi+SiSi/svc_structure_cantons.dat', dtype=int)

In [52]:
for idx_deem, idx_iza in zip(deem_idxs, iza_idxs):
    print(labels_iza[idx_iza], pcantons_iza[idx_iza], idx_deem+8000001, cantons_deem[idx_deem])
    
print(np.count_nonzero(pcantons_iza[iza_idxs] == cantons_deem[deem_idxs]) / len(deem_idxs))

GIS 3 8000282 3
LTJ 4 8046159 4
RTE 1 8048551 1
UEI 3 8053433 3
AFY 3 8054148 3
PHI 2 8056793 2
IWR 2 8060435 2
BEC 1 8067418 1
CHA 3 8068050 3
AFR 2 8068316 2
AWW 3 8072989 3
AEI 3 8077977 3
GME 2 8077978 2
SIV 2 8077983 2
ITE 2 8082737 2
SAS 3 8085224 3
HEU 2 8094207 2
LEV 2 8095118 2
MOR 2 8115524 2
FER 1 8115812 1
MRT 3 8119854 3
ZON 3 8123580 3
EZT 2 8132964 2
GOO 2 8158586 2
GON 2 8166073 2
PON 4 8168597 4
STI 2 8186325 2
IFO 4 8236886 4
ESV 2 8252730 2
IWV 2 8285528 2
NES 2 8285546 2
SAF 3 8295280 3
BOG 2 8298667 2
IFY 1 8306634 1
RHO 2 8306957 2
ASV 3 8307029 3
UOZ 3 8307096 3
KFI 3 8307701 3
FAU 2 8307996 2
ANA 4 8308045 4
SAV 3 8308796 3
POR 4 8309058 4
SAO 2 8309154 2
MEL 2 8311069 2
ISV 1 8316304 1
POS 2 8318264 2
TSC 2 8320020 2
SBE 3 8320023 3
OFF 2 8321330 2
AFV 3 8321600 3
ERI 2 8321602 2
AVL 3 8321605 3
EAB 2 8321606 2
AFX 3 8321918 3
AVE 2 8322677 2
LTL 2 8322711 2
ETR 3 8322713 3
MAZ 4 8323247 4
FRA 2 8323748 2
AFS 2 8323974 2
MSO 2 8324148 2
SFW 2 8324260 2
MEI 2 83